In [25]:
import numpy as np
import matplotlib.pyplot as plt
import sigmf
from sigmf import sigmffile, SigMFFile

## Load the dataset

In [26]:
filename = 'data/dataset'
sigFile = sigmffile.fromfile(filename,skip_checksum=True)
annotations = sigFile.get_annotations()
nSignals = len(annotations)
# TODO: For now, I'm assuming all the data has the same length
nSamps = int(sigFile.sample_count/nSignals)
# The input data is a tensor with shape nSignals x 2 x nSamps
x = np.zeros((nSignals,2,nSamps))
for iSignal,annotation in enumerate(annotations):
  # Start index of the current signal
  annotationStart = annotation[SigMFFile.START_INDEX_KEY]
  # Length of the current signal in samples
  annotationLength = annotation[SigMFFile.LENGTH_INDEX_KEY]
  # Data for the current signal
  samples = sigFile.read_samples(annotationStart,annotationLength)
  # Store the signal data in the input tensor
  # Note: Since the data is complex, we need to split it into real and imaginary
  # parts because neural networks have trouble handling complex data
  x[iSignal,0,:] = np.real(samples)
  x[iSignal,1,:] = np.imag(samples)
  


## Partition the data into training and testing sets

In [28]:
np.random.seed(0)
nTrain = int(0.5*nSignals)
nTest = nSignals - nTrain
# Since the input data is heavily structured, we need to shuffle the data to
# form the training and testing sets
trainIndex = np.random.permutation(range(0,nSignals))[0:nTrain]
# The test set is the complement of the training set
testIndex = list(set(range(0,nSignals)) - set(trainIndex))
xTrain = x[trainIndex,:,:]
xTest = x[testIndex,:,:]


## Convert the class labels to one-hot vectors

In [ ]:
# TODO: Implement me!